In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import seaborn as sns
from scipy.optimize import minimize

# Enable interactive plots


In [2]:
import warnings

warnings.filterwarnings('error', category=np.ComplexWarning)
# Turn all runtime warnings into errors
warnings.simplefilter("error", RuntimeWarning)
np.seterr(all='warn')  # Convert all NumPy warnings to Python warnings

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Sytem Model

## Channel Model

#### Distance matrices

![Location setup](../location.png)


In [3]:
def Coordinates_system(Nt, lt, Nr, lr, Na, Nb, d_ris, D, wavelength):

    # Spacing between antennas (in meters)
    antenna_spacing = wavelength/2  

    ######################################################################################################
    # Tx coordinates
    # x,y coordinates is zeros
    tx_antenna_x_coor = np.ones(Nt) * 0
    tx_antenna_y_coor = np.arange(Nt-1, -1, -1) * antenna_spacing + lt - ((Nt-1) * antenna_spacing)/2
    tx_antenna_z_coor = np.ones(Nt) * 0


    ######################################################################################################
    # Rx coordinates
    # x coordinate is D
    # y coordinate is zeros
    rx_antenna_x_coor = np.ones(Nr) * D
    rx_antenna_y_coor = np.arange(Nr-1, -1, -1) * antenna_spacing + lr - ((Nr-1) * antenna_spacing)/2
    rx_antenna_z_coor = np.ones(Nr) * 0


    ######################################################################################################
    # RIS coordinates
    # on x axis we have Na elements
    # on y axis we have Nb elements

    # RIS z coordinates are zero
    # RIS x coordinates 
    col = np.arange(0, Na, 1).reshape(-1, 1)
    col = np.tile(col, Nb) * antenna_spacing + d_ris - ((Na-1) * antenna_spacing)/2
    ris_elements_x_coor = col.reshape(-1,)

    ris_elements_y_coor = np.ones(Na*Nb) * 0

    # RIS y coordinates 
    col = np.arange(0, Nb, 1)
    col = np.tile(col, Na) * antenna_spacing - ((Nb-1) * antenna_spacing)/2
    ris_elements_z_coor = col.reshape(-1,)

    return [
        tx_antenna_x_coor,
        tx_antenna_y_coor,
        tx_antenna_z_coor,
        rx_antenna_x_coor,
        rx_antenna_y_coor,
        rx_antenna_z_coor,
        ris_elements_x_coor,
        ris_elements_y_coor,
        ris_elements_z_coor
    ]
    
# Caluating distance vectors to be used in calulating path losses
# generate N1xN2 size distance matrix
# i,j elements refer to distance between ith element in mat1 and jth element in mat2
def CalCulateDistance(N1, N2, mat1_x, mat1_y, mat1_z, mat2_x, mat2_y, mat2_z):
    distance_matrix = np.zeros((N1, N2))

    for n1 in range(N1):
        for n2 in range(N2):
            distance_matrix[n1,n2] = np.sqrt(
                (mat1_x[n1] - mat2_x[n2]) ** 2 +
                (mat1_y[n1] - mat2_y[n2]) ** 2 +
                (mat1_z[n1] - mat2_z[n2]) ** 2 
            )

    return distance_matrix


def GenerateDistanceMatrix(Nt, lt, Nr, lr, Na, Nb, d_ris, D, wavelength):
    N_ris = Na * Nb
    
    # Rx and Tx
    [
        tx_antenna_x_coor,
        tx_antenna_y_coor,
        tx_antenna_z_coor,
        rx_antenna_x_coor,
        rx_antenna_y_coor,
        rx_antenna_z_coor,
        ris_elements_x_coor,
        ris_elements_y_coor,
        ris_elements_z_coor
    ] = Coordinates_system(Nt, lt, Nr, lr, Na, Nb, d_ris, D, wavelength)
    
    distance_r_t = CalCulateDistance(Nr, Nt, 
        rx_antenna_x_coor, rx_antenna_y_coor, rx_antenna_z_coor,
        tx_antenna_x_coor, tx_antenna_y_coor, tx_antenna_z_coor
    )

    # Ris and Tx
    distance_ris_t = CalCulateDistance(N_ris, Nt, 
        ris_elements_x_coor, ris_elements_y_coor, ris_elements_z_coor,
        tx_antenna_x_coor, tx_antenna_y_coor, tx_antenna_z_coor
    )

    # RIS and Rx
    distance_r_ris = CalCulateDistance(Nr, N_ris, 
        rx_antenna_x_coor, rx_antenna_y_coor, rx_antenna_z_coor,
        ris_elements_x_coor, ris_elements_y_coor, ris_elements_z_coor
    )

    return [
        distance_r_t,
        distance_ris_t,
        distance_r_ris
    ]



In [4]:
# alpha
# Free space pathloss exponent of the direct/indirect link, whose value is influenced by the obstacle present.
def GenHLOS(distance_matrix, wavelength):
    return np.exp(-1j * 2 * np.pi * distance_matrix / wavelength)


# N1, N2 is dimensions of distance matrix used to generate HLOS
def GenHNOLOS(N1, N2, varaince = 1):
    real_part = np.random.normal(0, np.sqrt(varaince/2), (N1, N2))
    imag_part = np.random.normal(0, np.sqrt(varaince/2), (N1, N2))

    return (real_part + 1j * imag_part)


# calculate general H 
def Calculate_H_LOSNLOS(N1, N2, distance_matrix, wavelength, racian_factor):
    H_D_LOS = GenHLOS(distance_matrix, wavelength)
    H_D_NLOS = GenHNOLOS(N1, N2, 2)
    H_LOSNLOS = (np.sqrt(racian_factor) * H_D_LOS + H_D_NLOS)

    return H_LOSNLOS


# calculate H DIR (Direct channel between tx and rx)
# alpha value is influenced by the obstacle present
def Calculate_H_DIR(Nr, Nt, distance_matrix_r_t, wavelength, alpha_dir, D, lt, lr, racian_factor):
    d_note = np.sqrt((D ** 2) + ((lt-lr) ** 2))
    beta_DIR = ((4 * np.pi / wavelength) ** 2) * (d_note ** alpha_dir)
    
    H_DIR = np.sqrt(1/(beta_DIR * (racian_factor + 1))) * Calculate_H_LOSNLOS(Nr, Nt, distance_matrix_r_t, wavelength, racian_factor)
    return H_DIR


def Calculate_H1(N_ris, Nt, distance_matrix_ris_t, wavelength, racian_factor, beta_INDIR_inverse):
    return  np.sqrt(beta_INDIR_inverse/(racian_factor + 1)) * Calculate_H_LOSNLOS(N_ris, Nt, distance_matrix_ris_t, wavelength, racian_factor)


def Calculate_H2(Nr, N_ris, distance_matrix_r_ris, wavelength, racian_factor):
    return np.sqrt(1/(racian_factor + 1)) * Calculate_H_LOSNLOS(Nr, N_ris, distance_matrix_r_ris, wavelength, racian_factor)


# calculate H INDIR (Infirect Direct channel between tx and rx)
def Calculate_H_INDIR(H1, H2, thetas, beta_INDIR_inverse):
    F_theta = np.diag(thetas)
    H_INDIR = np.matmul(H2, np.matmul(F_theta, H1))

    return H_INDIR


def Calculate_H(HDIR, HINDIR):
    return HDIR + HINDIR

    

In [5]:
def Rate(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q):
    Z_1 = H_dir_1 + H2_1 @ np.diag(thetas_1) @ H1_1
    Z_2 = H_dir_2 + H2_2 @ np.diag(thetas_2) @ H1_2
    Z = Z_1 + Z_2
    Nr = H_dir_1.shape[0]
    
    rate = np.real(np.log(np.linalg.det(np.eye(Nr) + Z @ Q @ Z.conj().T))) / np.log(2)
    return rate


## Projected Gradient Method

In [6]:
def Calculate_Kappa(H_dir, H2, H1, Pt):
    if H_dir.all() == 0:
        return 10
    else :
        return 10 * max(1, 1/np.sqrt(Pt)) * np.sqrt( np.linalg.norm(H_dir)/ np.linalg.norm(np.matmul(H2, H1)))
    
    
def Calculate_LTheta(a, b, Pt):
  """
  Calculates the value of L_theta^2 based on equation (32).

  Args:
    a: The value of 'a'.
    b: The value of 'b'.
    Pt: The value of 'Pt'.

  Returns:
    The calculated value of L_theta^2.
  """

  term1 = 2 * a * (b**5) + 4 * (a**2) * (b**2)
  term2 = (a**3) * b + 2 * a * (b**7) + 8 * (a**2) * (b**4)
  term3 = 3 * (a**3) * (b**3) + (a**4) + 4 * (a**2) * (b**6)
  term4 = 2 * (a**3) * (b**5) + 4 * (a**4) * (b**2)
  term5 = 4 * (a**4) * (b**4)

  return term1 + term2 * Pt + term3 * (Pt**2) + term4 * (Pt**3) + term5 * (Pt**4)


def Calculate_LQ(a, b, Pt):
  """
  Calculates the value of L_Q^2 based on equation (33).

  Args:
    a: The value of 'a'.
    b: The value of 'b'.
    Pt: The value of 'Pt'.

  Returns:
    The calculated value of L_Q^2.
  """

  term1 = (a**2) * (b**2) + (b**8) + 2 * a * (b**5)
  term2 = 2 * (a**2) * (b**4) + (a**3) * b + 2 * a * (b**7)
  term3 = (a**2) * (b**6) + 3 * (a**3) * (b**3)
  term4 = 2 * (a**3) * (b**5)

  return term1 + term2 * Pt + term3 * (Pt**2) + term4 * (Pt**3)


def L_constant(H_dir_bar, H1_bar, H2, kappa, Pt_bar):
    U_h1_bar, S_h1_bar, Vh_h1_bar = np.linalg.svd(H1_bar)
    U_h2, S_h2, Vh_h2 = np.linalg.svd(H2)
    U_H_dir_bar, S_H_dir_bar, Vh_H_dir_bar = np.linalg.svd(H_dir_bar)
    
    
    lambda_max_h1_bar = np.max(S_h1_bar)
    lambda_max_h2 = np.max(S_h2)
    lambda_max_H_dir_bar = np.max(S_H_dir_bar)
    
    a = lambda_max_h1_bar * lambda_max_h2
    b = lambda_max_H_dir_bar + lambda_max_h1_bar * lambda_max_h2 / kappa

    L_2_theta = Calculate_LTheta(a,b, Pt_bar)
    L_2_Q = Calculate_LQ(a, b, Pt_bar)

    L = np.sqrt(max(L_2_theta, L_2_Q))
    return L


def water_fill(Pt, vect_in):
    vect_in = np.real(vect_in).flatten()
    sort_idx = np.argsort(vect_in)[::-1]
    sort_val = vect_in[sort_idx]
    n = len(vect_in)
    for i in range(n, 0, -1):
        water_level = (np.sum(sort_val[:i]) - Pt) / i
        di = sort_val[:i] - water_level
        if np.all(di >= 0):
            break
    vect_out = np.zeros_like(vect_in)
    vect_out[sort_idx[:i]] = di
    return vect_out

def cov_mat_proj_modified(Qold, Pt):
    D, U = np.linalg.eig(Qold)
    Dnew = water_fill(Pt, D)
    return U @ np.diag(Dnew) @ U.conj().T

def project_onto_unit_circle(x):
    return x / np.abs(x)

def grad_cov(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q):
    Z = H_dir_1 + H2_1 @ np.diag(thetas_1) @ H1_1 + H_dir_2 + H2_2 @ np.diag(thetas_2) @ H1_2
    Nr = H_dir_1.shape[0]
    
    return Z.conj().T @ np.linalg.inv(np.eye(Nr) + Z @ Q @ Z.conj().T) @ Z

def grad_RIS(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q):
    Z = H_dir_1 + H2_1 @ np.diag(thetas_1) @ H1_1 + H_dir_2 + H2_2 @ np.diag(thetas_2) @ H1_2
    Nr = H_dir_1.shape[0]
    
    return np.diag(H2_1.conj().T @ np.linalg.inv(np.eye(Nr) + Z @ Q @ Z.conj().T) @ Z @ Q @ H1_1.conj().T)


def PGM(max_iterations, H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, Nr, D, initial_Q, initial_thetas_1, initial_thetas_2, noise_power, Pt):
    H_dir_2 = H_dir_2 * 0
    delta = 1e-5
    rho = 0.5
    stepsize = 10000
    thetas_1 = initial_thetas_1
    thetas_2 = initial_thetas_2
    Q = initial_Q
    
    kappa = max(Calculate_Kappa(H_dir_1, H2_1, H1_1, Pt), Calculate_Kappa(H_dir_2, H2_2, H1_2, Pt))
    
    # Scaling data
    Q = (kappa ** 2) * Q
    thetas_1 = thetas_1 / kappa
    thetas_2 = thetas_2 / kappa
    
    H_dir_1 = H_dir_1 / (kappa * np.sqrt(noise_power))
    H1_1 = H1_1 * (np.sqrt(1 / noise_power))
    
    H_dir_2 = H_dir_2 / (kappa * np.sqrt(noise_power))
    H1_2 = H1_2 * (np.sqrt(1 / noise_power))

    # inproved convergence rate by data scaling
    # section 3.D
    initial_rate = Rate(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
    rate = initial_rate
    rates = np.zeros(max_iterations+1)
    rates[0] = rate

    # algo run for max_iteration times or rate convergers
    for i in range(1, max_iterations+1):

        del_Q        = grad_cov(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
        del_thetas_1 = grad_RIS(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
        del_thetas_2 = grad_RIS(H_dir_2, H1_2, H2_2, H_dir_1, H1_1, H2_1, thetas_1, thetas_2, Q)
        
        for _ in range(31):
            Q_new = Q + stepsize * del_Q
            Q_new = cov_mat_proj_modified(Q_new, Pt * (kappa ** 2))

            thetas_new_1 = thetas_1 + stepsize * del_thetas_1
            thetas_new_1 = project_onto_unit_circle(thetas_new_1) / kappa
            thetas_new_2 = thetas_2 + stepsize * del_thetas_2
            thetas_new_2 = project_onto_unit_circle(thetas_new_2) / kappa

            rate_new = Rate(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_new_1, thetas_new_2, Q_new)

            if (rate_new - rate >= delta * (np.linalg.norm(Q_new - Q) ** 2 + np.linalg.norm(thetas_new_1 - thetas_1) ** 2 + np.linalg.norm(thetas_new_2 - thetas_2) ** 2)) or (stepsize < 1e-4):
                thetas_1 = thetas_new_1
                thetas_2 = thetas_new_2
                Q = Q_new
                rate = rate_new
                break
            else:
                stepsize *= rho

        rates[i] = rate
        
        
        
    
    return [rates, 1, 1]

        

In [7]:

def PGM_Quantized_last_step(max_iterations, H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, Nr, D, initial_Q, initial_thetas_1, initial_thetas_2, noise_power, Pt, theta_quant_size):
    delta = 1e-5
    rho = 0.5
    stepsize = 10000
    thetas_1 = initial_thetas_1
    thetas_2 = initial_thetas_2
    Q = initial_Q
    
    kappa = max(Calculate_Kappa(H_dir_1, H2_1, H1_1, Pt), Calculate_Kappa(H_dir_2, H2_2, H1_2, Pt))
    
    # Scaling data
    Q = (kappa ** 2) * Q
    thetas_1 = thetas_1 / kappa
    thetas_2 = thetas_2 / kappa
    
    H_dir_1 = H_dir_1 / (kappa * np.sqrt(noise_power))
    H1_1 = H1_1 * (np.sqrt(1 / noise_power))
    
    H_dir_2 = H_dir_2 / (kappa * np.sqrt(noise_power))
    H1_2 = H1_2 * (np.sqrt(1 / noise_power))

    # inproved convergence rate by data scaling
    # section 3.D
    initial_rate = Rate(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
    rate = initial_rate
    rates = np.zeros(max_iterations+1)
    rates[0] = rate

    # algo run for max_iteration times or rate convergers
    for i in range(1, max_iterations+1):

        del_Q        = grad_cov(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
        del_thetas_1 = grad_RIS(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
        del_thetas_2 = grad_RIS(H_dir_2, H1_2, H2_2, H_dir_1, H1_1, H2_1, thetas_1, thetas_2, Q)
        
        for _ in range(31):
            Q_new = Q + stepsize * del_Q
            Q_new = cov_mat_proj_modified(Q_new, Pt * (kappa ** 2))

            thetas_new_1 = thetas_1 + stepsize * del_thetas_1
            thetas_new_1 = project_onto_unit_circle(thetas_new_1) / kappa
            thetas_new_2 = thetas_2 + stepsize * del_thetas_2
            thetas_new_2 = project_onto_unit_circle(thetas_new_2) / kappa

            rate_new = Rate(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_new_1, thetas_new_2, Q_new)

            if (rate_new - rate >= delta * (np.linalg.norm(Q_new - Q) ** 2 + np.linalg.norm(thetas_new_1 - thetas_1) ** 2 + np.linalg.norm(thetas_new_2 - thetas_2) ** 2)) or (stepsize < 1e-4):
                thetas_1 = thetas_new_1
                thetas_2 = thetas_new_2
                Q = Q_new
                rate = rate_new
                break
            else:
                stepsize *= rho

        rates[i] = rate
    
    angle_rad_1 = np.angle(thetas_1 * kappa, deg=False)
    angles_rad_2pi_1 = angle_rad_1 + (angle_rad_1 < 0) * 2 * np.pi
    quantized_angles_1 = quantize_angles(angles_rad_2pi_1, theta_quant_size)
    thetas_1 = np.exp(1j * quantized_angles_1) / kappa
    
    angle_rad_2 = np.angle(thetas_2 * kappa, deg=False)
    angles_rad_2pi_2 = angle_rad_2 + (angle_rad_2 < 0) * 2 * np.pi
    quantized_angles_2 = quantize_angles(angles_rad_2pi_2, theta_quant_size)
    thetas_2 = np.exp(1j * quantized_angles_2) / kappa

    rate = Rate(H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, thetas_1, thetas_2, Q)
    rates[max_iterations] = rate
    
    return [rates, 1, 1]



def quantize_angles(angles, n):
    """
    Quantizes the angles between 0 and 2π into n levels.

    Args:
    angles (numpy array): Array of angles between 0 and 2π.
    n (int): Number of quantization levels.

    Returns:
    numpy array: Quantized angles.
    """
    
    # Generate quantization levels
    quant_levels = np.linspace(0, 2 * np.pi, n, endpoint=False)
    
    # Find the closest quantization level for each angle
    quantized_angles = np.array([quant_levels[np.argmin(np.abs(quant_levels - angle))] for angle in angles])
    
    return quantized_angles

## Simulation parameter

In [8]:
def GenerateChannels(Nt, lt, Nr, lr, D, d_ris, Na, Nb, wavelength, alpha_dir, racian_factor):
    d1 = np.sqrt(d_ris ** 2 + lt ** 2)
    d2 = np.sqrt((D - d_ris) ** 2 + lr ** 2)
    beta_INDIR_inverse = ((wavelength**4)/(256 * (np.pi ** 2))) * (((lt/d1 + lr/d2)**2) / ((d1*d2)**2))

    N_ris = Na * Nb
    Nl = N_ris
    
    [
        distance_r_t,
        distance_ris_t,
        distance_r_ris
    ] = GenerateDistanceMatrix(Nt, lt, Nr, lr, Na, Nb, d_ris, D, wavelength)

    initial_thetas = np.ones(N_ris, dtype=np.complex64)
    
    
    H_dir = Calculate_H_DIR(Nr, Nt, distance_r_t, wavelength, alpha_dir, D, lt, lr, racian_factor)
    H1 = Calculate_H1(N_ris, Nt, distance_ris_t, wavelength, racian_factor, beta_INDIR_inverse)
    H2 = Calculate_H2(Nr, N_ris, distance_r_ris, wavelength, racian_factor)

    return [initial_thetas, H_dir, H1, H2]

    

In [9]:
# transmit antennas 
Nt = 8
# receiving antennas
Nr = 4

# both tx and rx antennas are placed in uniform linear arrays on vertical walls paraller to each other on a wall
D = 500

# wavelenght and frequency of operation
frequency = 2 * (10 ** 9)
wavelength = (3 * (10 ** 8))/frequency
antenna_spacing = wavelength/2

# inter antenna seperation distance
st = wavelength/2
sr = wavelength/2
# seperation between center of adjacent RIS elements in both dimension is
s_ris = wavelength / 2

# power in db = 10log10(power(watts))

# noise power (untis not mentioned in paper)
# -120 db
noise_power = (10 ** (-12))
# total average power transmit value Pt
# 0 db
Pt = 1

# path loss exponent for direct link influcend by obstacle present
alpha_dir = 3

# racian factor
racian_factor = 1

# backtracking line search parameters
L_note = 10**4
delta = 10 ** (-5)
row = 1/2
min_step_size = 10**(-4)
channel_iterations = 10

# dirs should be placed in vicinity of tx or rx
# if near tx Dris = 40
# if near rx Dris = D - 40

# distance between midpoint of RIS and the placne containing tx antennas
d_ris_1 = 40
d_ris_2 = 40
# distance between mid point of tx antenna arraya to plane contining RIS is 
lt_1 = 20
lt_2 = 30
# distance between rx antenna to plane containing RIS
lr_1 = 100
lr_2 = 80

Na = 15
Nb = 15

number_of_itr = 500

In [10]:
max_iterations = 500
rate_average = np.zeros(max_iterations+1)
average_over = 10
step_boost = 0
intial_Q = (Pt/Nt) * np.eye(Nt)

for i in range(average_over):
    # if(i % 10 !):
    print(f"PGM for {i+1}th channel")
    [thetas_1, H_dir_1, H1_1, H2_1] = GenerateChannels(Nt, lt_1, Nr, lr_1, D, d_ris_1, Na, Nb, wavelength, alpha_dir, racian_factor)
    [thetas_2, H_dir_2, H1_2, H2_2] = GenerateChannels(Nt, lt_2, Nr, lr_2, D, d_ris_2, Na, Nb, wavelength, alpha_dir, racian_factor)
    [rates, _, _] = PGM(
        max_iterations, 
        H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, 
        Nr, D, intial_Q, thetas_1, thetas_2, noise_power, Pt
    )
    rate_average += rates / average_over
    

# rate_average = rate_average 

    

PGM for 1th channel
PGM for 2th channel
PGM for 3th channel
PGM for 4th channel
PGM for 5th channel
PGM for 6th channel
PGM for 7th channel
PGM for 8th channel
PGM for 9th channel
PGM for 10th channel


In [ ]:
# Create the line plot
normal_rate_fig = go.Figure(data=go.Scatter(
    x=np.log10(np.arange(0, len(rate_average))+1),
    y=rate_average, 
    mode='lines+markers',  # Line + Markers
    marker=dict(size=4, color='red'),  # Marker style
    line=dict(color='blue', width=1),  # Line style
    name='Line with Markers'
)) 

# Add titles and labels
normal_rate_fig.update_layout(
    title='Achivable Rate (dirs 1 = 40, dirs 2 = 460)',
    xaxis_title='Iteration number',
    yaxis_title='Rate (bit/s/Hz)'  # Optional: Dark theme, change if needed
)

In [48]:

max_iterations = 500
average_over = 10
theta_step_size = 10

max_rates = np.zeros(10)
quantized_rate = np.zeros(10)

for theta_step_size in range(1, 11):
    print(f"step size {theta_step_size}")
    qrate_average = np.zeros(max_iterations+1)

    for i in range(average_over):
        [thetas_1, H_dir_1, H1_1, H2_1] = GenerateChannels(Nt, lt_1, Nr, lr_1, D, d_ris_1, Na, Nb, wavelength, alpha_dir, racian_factor)
        [thetas_2, H_dir_2, H1_2, H2_2] = GenerateChannels(Nt, lt_2, Nr, lr_2, D, d_ris_2, Na, Nb, wavelength, alpha_dir, racian_factor)
        [rates, _, _] = PGM_Quantized_last_step(
            max_iterations, 
            H_dir_1, H1_1, H2_1, H_dir_2, H1_2, H2_2, 
            Nr, D, intial_Q, thetas_1, thetas_2, noise_power, Pt,
            2 ** theta_step_size
        )
        
        qrate_average += rates / average_over
        
    max_rates[theta_step_size-1] = qrate_average[-2]
    quantized_rate[theta_step_size-1] = qrate_average[-1]

    
    

step size 1
step size 2
step size 3
step size 4
step size 5
step size 6
step size 7
step size 8
step size 9
step size 10


In [49]:
# Create traces for real part
mixfigure = go.Figure()
mixfigure.add_trace(go.Scatter(x=np.arange(1, 11), y=max_rates, mode='lines+markers', name='Non qantized'))
mixfigure.add_trace(go.Scatter(x=np.arange(1, 11), y=quantized_rate, mode='lines+markers', name='Quantized'))

# Add titles and labels
mixfigure.update_layout(
    title='Indirect link length (dirs = 40)',
    xaxis_title='Iteration number',
    yaxis_title='Rate (bit/s/Hz)',
    template='plotly_dark',  # Optional: Dark theme, change if needed
)